In [1]:
import pandas as pd

df_monthly=pd.read_csv("./data/monthly_csv.csv")

# 시계열 자료를 년도와 월로 나눠 시간 순으로 정렬
df_monthly['Year'] = pd.DatetimeIndex(df_monthly['Date']).year
df_monthly['Month'] = pd.DatetimeIndex(df_monthly['Date']).month
df_monthly.set_index(["Date"])

,Price,Year,Month
Date,,,
1950-01,34.730,1950,1
1950-02,34.730,1950,2
1950-03,34.730,1950,3
1950-04,34.730,1950,4
1950-05,34.730,1950,5
...,...,...,...
2020-03,1593.764,2020,3
2020-04,1680.030,2020,4
2020-05,1715.697,2020,5


In [2]:
import numpy as np

price=df_monthly["Price"].to_numpy()

train_x=[]
train_y=[]

test_x=price[-10:-1]
test_y=price[-1]

for i in range(price.shape[0]-10):
    #print(price[i:i+9])
    train_x.append(price[i:i+9])
    #print(price[i+9])
    train_y.append(price[i+9])
    
train_x=np.array(train_x)
train_y=np.array(train_y)
train_x=np.reshape(train_x,(-1,9,1))

print(train_x.shape,train_y.shape)
print(test_x.shape,test_y.shape)

(837, 9, 1) (837,)
(9,) ()


In [3]:
import tensorflow as tf
from tensorflow.keras import layers

# 모델 생성
inputs=tf.keras.Input(shape=(9,1),name="input")

# 기존 LSTM은 이전 데이터의 time step들이 이후 데이터 time step에 영향을 줄 것인지에 대해 가정한다면
# Bidirectional LSTM은 이후 데이터의 time step가 이전 데이터의 time step에 영향을 줄것이지에 대한 가정이다
# Bidirectional LSTM은 두개의 LSTM을 연결해 사용한다

#lstw1=layers.LSTM(6,return_sequences=True)(inputs)
#lstw2=layers.LSTM(6,return_sequences=True)(lstw1)
bilstm1=layers.Bidirectional(layers.LSTM(6,return_sequences=True))(inputs)
bilstm2=layers.Bidirectional(layers.LSTM(6))(bilstm1)
di=layers.Dense(6,activation="relu")(bilstm2)
outputs=layers.Dense(1)(di)

model=tf.keras.Model(inputs=inputs,outputs=outputs)  

# 모델 학습과정 설정 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
             loss=tf.keras.losses.MeanSquaredError(),
             metrics=["mse"])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 9, 1)]            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 9, 12)             384       
_________________________________________________________________
bidirectional_1 (Bidirection (None, 12)                912       
_________________________________________________________________
dense (Dense)                (None, 6)                 78        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 1,381
Trainable params: 1,381
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.fit(train_x,train_y,epochs=20,batch_size=100)

Epoch 1/20
9/9 [==============================] - 11s 13ms/step - loss: 366517.2000 - mse: 366517.2000
Epoch 2/20
9/9 [==============================] - 0s 8ms/step - loss: 395471.6813 - mse: 395471.6813
Epoch 3/20
9/9 [==============================] - 0s 9ms/step - loss: 378981.6531 - mse: 378981.6531
Epoch 4/20
9/9 [==============================] - 0s 9ms/step - loss: 370536.1281 - mse: 370536.1281
Epoch 5/20
9/9 [==============================] - 0s 9ms/step - loss: 354691.4875 - mse: 354691.4875
Epoch 6/20
9/9 [==============================] - 0s 9ms/step - loss: 343425.3219 - mse: 343425.3219
Epoch 7/20
9/9 [==============================] - 0s 10ms/step - loss: 354381.4781 - mse: 354381.4781
Epoch 8/20
9/9 [==============================] - 0s 10ms/step - loss: 363931.4219 - mse: 363931.4219
Epoch 9/20
9/9 [==============================] - 0s 9ms/step - loss: 349460.0469 - mse: 349460.0469
Epoch 10/20
9/9 [==============================] - 0s 9ms/step - loss: 337693.1813 - ms

In [5]:
# 모델 예측
model.predict(np.reshape(test_x,(-1,9,1)))

array([[255.83824]], dtype=float32)

In [ ]:
# https://pozalabs.github.io/blstm/